In [1]:
! pip install folium

     ---------------------------------------- 0.0/102.3 kB ? eta -:--:--
     -------------- ---------------------- 41.0/102.3 kB 653.6 kB/s eta 0:00:01
     --------------------------------- --- 92.2/102.3 kB 871.5 kB/s eta 0:00:01
     ------------------------------------ 102.3/102.3 kB 836.7 kB/s eta 0:00:00
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached charset_normalizer-3.1.0-cp310-cp310-win_amd64.whl (97 kB)


In [1]:
import numpy as np
import pandas as pd
import folium

In [2]:
df = pd.read_csv("data/2023/Divvy_Trips_2023_full.csv.gz", compression="gzip")

# Общая информация о датасете

In [3]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639424 entries, 0 to 639423
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             639424 non-null  object 
 1   rideable_type       639424 non-null  object 
 2   started_at          639424 non-null  object 
 3   ended_at            639424 non-null  object 
 4   start_station_name  551320 non-null  object 
 5   start_station_id    551188 non-null  object 
 6   end_station_name    546408 non-null  object 
 7   end_station_id      546267 non-null  object 
 8   start_lat           639424 non-null  float64
 9   start_lng           639424 non-null  float64
 10  end_lat             638998 non-null  float64
 11  end_lng             638998 non-null  float64
 12  member_casual       639424 non-null  object 
dtypes: float64(4), object(9)
memory usage: 63.4+ MB


## Работа с данными и построение карты

In [5]:
#Выбор нужных колонок датасета - координаты, название и id станций

# Данные начальных станций
station_names = df[['start_station_name']]
station_names = station_names.rename(columns={'start_station_name': 'station_name'})

# Данные конечных станций
station_end_names = df[['end_station_name']]
station_end_names = station_end_names.rename(columns={'end_station_name': 'station_name'})

In [6]:
# Склейка данных в 1 датасет
station_names = pd.concat([station_names, station_end_names])
station_names = station_names.drop_duplicates().dropna()
df.rename(columns={'end_station_name': 'station_name'}, inplace=True)
# Убираем дупликаты по имени
coord = station_names.join(df.set_index('station_name')[['start_lng', 'start_lat']], on='station_name').drop_duplicates(subset=['station_name'], keep='first')
coord.rename(columns={'start_lng': 'lng', 'start_lat': 'lat'}, inplace=True)


In [11]:
# Удаление дублей и пустых значений
coord = coord.drop_duplicates()
coord = coord.dropna()
# Колонка имени больше не нужна
coord = coord.drop(columns=['station_name'])

In [12]:
coord.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1120 entries, 0 to 617720
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lng     1120 non-null   float64
 1   lat     1120 non-null   float64
dtypes: float64(2)
memory usage: 26.2 KB


In [18]:
coord = coord.values.tolist()

In [19]:
from folium.plugins import MarkerCluster


chicago = folium.Map(location=[41.85, -87.65], zoom_start=11)

marker_cluster = MarkerCluster().add_to(chicago)



for i in coord:
    folium.CircleMarker(location=(i[1], i[0]), fill_color='#990066', color="gray", fill_opacity = 0.9).add_to(marker_cluster)

chicago